## IDEA FOR CACHING: Check overlapping between the doc-ids of the query at the current turn and the doc-ids of queries of the previout turns

In [13]:
#1. loading data: load the first k (e.g. k=10) docs for each utterance in a dictionary

map_utt_docs = {} #dictionary: key = utt_id, value = list of top-k results

max_k = 20
k = 0

#local
#path = "/Users/idamele/Desktop/CAST_qrels/"
#server
path = "../data/CAST_qrels/"
with open(path+"qrels-docs.2019.txt", 'r') as f:
#with open(path+"qrels-docs.2020.txt", 'r') as f:
#with open(path+"qrels-docs.2021.txt", 'r') as f:
    for line in f:
        line_fields = line.rstrip().split(" ")
        utt_id = line_fields[0]
        doc_id = line_fields[2]
        if k < max_k:
            k = k + 1 
            prev_utt_id = utt_id
            if utt_id in map_utt_docs: 
                map_utt_docs[utt_id].append(doc_id)
            else:
                map_utt_docs[utt_id] = [doc_id]
        else:           
            if utt_id == prev_utt_id:
                # after top-k we do not consider them
                prev_utt_id = utt_id
            else:
                #new conversation, insert first document in the map
                k = 1 
                map_utt_docs[utt_id] = [doc_id]
            
        

In [14]:
map_utt_docs["31_2"]

['CAR_08489902963b8b03a75ebfc2afad422660c1cc71',
 'CAR_0baad9b74477434c04136d8c994af6bcb3aed7ed',
 'CAR_116d829c4c800c2fc70f11692fec5e8c7e975250',
 'CAR_1244f628b92ca5ff3bc28191a370726561154166',
 'CAR_16b23027e80fe73ba67f32995c488f89b12d0ac9',
 'CAR_172e16e89ea3d5546e53384a27c3be299bcfe968',
 'CAR_1c93ef499a0c2856c4a857b0cb4720c380dda476',
 'CAR_1f03ba408f108f9edc5ef4e9958cf0c1bab66da7',
 'CAR_2174ad0aa50712ff24035c23f59a3c2b43267650',
 'CAR_26f24506a2688c24da3c0a6158228dfdd2d13b67',
 'CAR_2dc597ac2fc10917a752552bc335e6ac1aedc3f0',
 'CAR_393cb4e18a9d30018e843c4d37c564272ec5fa6f',
 'CAR_462db9a569840533b644a7eb3e2557a23ae8204b',
 'CAR_4fc9cfd310dfb8eaf3fbfa60723ec5b8072b876d',
 'CAR_55a95284bc782f3da5d3651a9c0e6f4df0ba7458',
 'CAR_55f68f0a33f49f6015035f70d8685e293389c9d6',
 'CAR_5afea520c45103a5637ed021a5129766d37341bc',
 'CAR_5b50a00d9e1bd1d4ac3a973a656c1b20ab96ec1e',
 'CAR_63e2ffba97498caee83a1f9c36e2cb5f9700df9d',
 'CAR_66e78c6918b07df45d2e26ebaba74baaeb0c0da0']

In [15]:
#functions to check the intersection between top-k results of current turn and top-k results of the previous turns 

def intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

def compute_overapping_documents(conv_id, curr_turn, f):
    
    total_overlapping = 0

    docs_current_turn =  map_utt_docs[str(conv_id)+"_"+str(curr_turn)]
    #print("current turn: "+str(curr_turn))
    #print(docs_current_turn)

    docs_so_far = {}
    for turn_id in range(1, curr_turn):
        if str(conv_id)+"_"+str(turn_id) in map_utt_docs:
            docs_so_far = map_utt_docs[str(conv_id)+"_"+str(turn_id)]
            #print("so_far: "+str(turn_id))
            #print(docs_so_far)
            #print(intersection(docs_so_far, docs_current_turn))
            #print(len(intersection(docs_so_far, docs_current_turn)))
            total_overlapping += len(intersection(docs_so_far, docs_current_turn))
        else:
            f.write(str(conv_id)+"_"+str(turn_id)+" not found\n")

    f.write(str(conv_id)+"_"+str(curr_turn)+"\t"+str(total_overlapping)+"\n")


def compute_overapping_documents_set(conv_id, curr_turn, f):
    
    total_overlapping = 0

    docs_current_turn =  map_utt_docs[str(conv_id)+"_"+str(curr_turn)]
    #print("current turn: "+str(curr_turn))
    #print(docs_current_turn)

    docs_so_far = set()
    for turn_id in range(1, curr_turn):
        if str(conv_id)+"_"+str(turn_id) in map_utt_docs:
            my_list = map_utt_docs[str(conv_id)+"_"+str(turn_id)]
            for el in my_list:                
                docs_so_far.add(el)
                
            #print("so_far: "+str(turn_id))
            #print(docs_so_far)
            #print(intersection(docs_so_far, docs_current_turn))
            #print(len(intersection(docs_so_far, docs_current_turn)))
           
        else:
            f.write(str(conv_id)+"_"+str(turn_id)+" not found\n")
    
    total_overlapping = len(intersection(docs_so_far, docs_current_turn))
    f.write(str(conv_id)+"_"+str(curr_turn)+"\t"+str(total_overlapping)+"\n")



In [16]:
#2. main: to run after loading data

#local
#path = "/Users/idamele/Desktop/CAST_qrels/results/"
#server
path = "../data/proof_of_concepts_preliminary_results/qrels/top20/"
with open(path+"overalapping-docs.2019.txt", 'w') as f:
#with open(path+"overalapping-docs.2020.txt", 'w') as f:
#with open(path+"overalapping-docs.2021.txt", 'w') as f:
    for el in map_utt_docs:
        conv_id = int(el.split("_")[0])
        curr_turn = int(el.split("_")[1])
        compute_overapping_documents(conv_id, curr_turn, f)



In [17]:
#2. main (with sets, no duplicates): to run after loading data

path = "../data/proof_of_concepts_preliminary_results/qrels/top20/"
with open(path+"overalapping-docs.2019_set.txt", 'w') as f:
#with open(path+"overalapping-docs.2020_set.txt", 'w') as f:
#with open(path+"overalapping-docs.2021_set.txt", 'w') as f:
    for el in map_utt_docs:
        conv_id = int(el.split("_")[0])
        curr_turn = int(el.split("_")[1])
        compute_overapping_documents_set(conv_id, curr_turn, f)